# <font color=blue><div align="center">Projet Karhoo : Métaheuristique</div></font>

## Importation des packages

In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import random

import copy
# Module csv
import csv

#Module json
import json

## Importation des datas

### 1- Données journalières

In [2]:
# Small_sized_data : dict
small_sized_data = "C:/Users/Zineb/Downloads/day_data.json"#insérer le chemin du fichier

# Chargement des données: 
with open(small_sized_data) as SmallSizedData:
    reader = json.load(SmallSizedData)
    bookings = reader["bookings"]
    shifts = reader["shifts"]

### 2- Identification entre les nom des stations et notre nomenclature


In [3]:
Stations_demandes={0:'s0'} #dictionaire qui lie chaque noeud avec la station réelle 
id_stations={}
Nombre_de_commandes = n= 25 
Nombres_de_shifts = 8
for i in range (Nombre_de_commandes):
    Stations_demandes[i+1]=bookings[i]['jobs'][0]['station']
    id_stations[i+1]=bookings[i]['jobs'][0]["id"]
    Stations_demandes[n+i+1]=bookings[i]['jobs'][1]['station']
    id_stations[n+i+1]=bookings[i]['jobs'][1]["id"]
    bookings[i]['id']=i+1
    bookings[i]['jobs'][0]['station']=i+1
    bookings[i]['jobs'][1]['station']=Nombre_de_commandes+i+1
Stations_demandes[51]='s51'

### 2- Temps de trajets entre les noeuds

In [4]:
travel_time = "C:/Users/Zineb/Downloads/travel_times4.csv"  #insérer le chemin du fichier

Time = {}

number_of_s = 52

with open(travel_time) as TravelTime:
    reader = csv.DictReader(TravelTime)
    for row in reader:
        b = int(row['station'])
        for sId in range(0, number_of_s +1):
            Time[(b,sId)] = float(row[f's{sId}'])
            
time = {}
stations = range(52)
for i in stations:
    for j in stations:
        d = int(Stations_demandes[i][1:])
        a = int(Stations_demandes[j][1:])
        time[(i, j)] = Time[(d, a)]


## Préparation des datas

### 1- Dictionnaire des charges

In [5]:
#nombre de commandes
n_req=25 
n_shift=8

#dict des charges:
load=[0,]
for r in range(n_req):
    a=int(bookings[r]["passengers"])
    load.append(a)
for r in range(n_req):
    a=int(bookings[r]["passengers"])
    load.append(-a)
load.append(0)
#print(load)

### 2- Liste des durées maximales autorisées

In [6]:
maximumDuration=[np.inf]
for i in range(n_req):
    m=bookings[i]["maximumDuration"]
    maximumDuration.append(m)


### 3- Dictionnaire des fenêtres de temps

In [7]:
#dict des time window 
e={}
l={}
for i in range (25):
    E=bookings[i]['jobs'][0]['timeWindowBeginDate']
    L=bookings[i]['jobs'][0]['timeWindowEndDate']
    e[i+1]=E
    l[i+1]=L
for i in range (25):
    E=bookings[i]['jobs'][1]['timeWindowBeginDate']
    L=bookings[i]['jobs'][1]['timeWindowEndDate']   
    e[n_req+i+1]=E
    l[n_req+i+1]=L

e[0]=min(min(e[i]-time[0,i] for i in range(1,26)),min(shifts[k]["jobs"][0]["timeDate"] for k in range(8)))
e[51]=min(min(e[i]-time[51,i] for i in range(1,26)),min(shifts[k]["jobs"][0]["timeDate"] for k in range(8)))
l[0]=max(max(l[i]-time[i,0] for i in range(26,51)),max(shifts[k]["jobs"][1]["timeDate"] for k in range(8)))
l[51]=max(max(l[i]-time[i,51] for i in range(26,51)),max(shifts[k]["jobs"][1]["timeDate"] for k in range(8)))
#print(e[0],e[51],l[0],l[51])

## solution initiale taha

# Construction de la solution réalisable

Nous allons tout d'abord construire une solution qui respecte tous les contraintes mais avec ne réponds pas à toutes les damandes:

- Chaque route doit commencer et finir aux depôts,
- Pour chaque commande le noeud $i$ et $n+i$ sont attribués au même chauffeur.

## Construction de la solution initiale

Nous allons dans un premier temps trier la liste des chauffeurs selon les horraires de départs, ceci ne facilitera l'affectation des premiére courses aux chauffeurs et de garantir que les marges de temps sont respecté et que les chauffeurs n'ont pas des temps d'atttente trés grand.
<section> Remarque : Malgré que nous n'avons pas de contrainte sur le temps d'attente des chauffeurs mais la minimisation de ce dernier nous permettera de minimiser le coût total de la course.

In [8]:
#Trie des début de shifts de chaque chauffeur par ordre décroissant
def classementChauffeurs(n):
    shift = {}
    for i in range(n):
        shift[i+1] = shifts[i]["jobs"][0]["timeDate"]
    L = list()
    for i in shift.values():
        L.append(i)
    L.sort(reverse = True)
    shift_trie = {}
    for i in L:
        for cle in shift.keys():
            if i == shift[cle]:
                shift_trie[cle] = i
    return shift_trie



In [9]:
def listeBookings(n):
    book = {}
    for i in range(n):
        book[i+1] = bookings[i]["jobs"][0]["timeWindowBeginDate"]

    return book

In [10]:
#print(listeBookings(n_req))

In [11]:
def listeBookings2(n):
    book2 = {}
    for i in range(n):
        book2[i+1] = bookings[i]["jobs"][1]["timeWindowEndDate"]
        
    return book2

In [12]:
#print(listeBookings2(n_req))

In [13]:
def initialisation(shift, bookings):
    
    book = copy.copy(bookings)

    nb_pick = len(book)           #Dernier pickup
    nb_noeuds = nb_pick*2 + 1     #Dernier dropoff
    
    Chauffeurs = {}
    
    for p in range(1,len(shift)+1):   #Création des listes des noeuds à visiter pour chaque chauffeur 
        Chauffeurs[p] = []
    
    
    for h in shift:
        b = 1000000 
        maxi = 0
        for k in book:
            if book[k] > maxi:
                maxi = book[k]     #Détermination du dernier pickup (en temps)
        if maxi==0:
            break
        for i in book:
            delta_1 = book[i] - shift[h] #Calcul de la différence de temps entre le départ du chauffeur et le pickup
            if delta_1 > time[h,i]:              #Cette différence doit-être positive pour pas qu'il puisse remonter le temps
                if b > delta_1 :
                    b = delta_1 
                    noeud = i
                    
        
        Chauffeurs[h].append(0)             #Ajout du noeud 0
        Chauffeurs[h].append(noeud)
        Chauffeurs[h].append(noeud+nb_pick)
        Chauffeurs[h].append(nb_noeuds)
        
        for m in Chauffeurs[h]:                 #Elimination des noeuds attribués dans la liste des pickup
            if m != 0:
                if m in range(1,nb_pick+1):
                    del book[m]
        
        
    return Chauffeurs

In [14]:
s1 = initialisation(classementChauffeurs(n_shift),listeBookings(n_req))
print(s1)

{1: [0, 19, 44, 51], 2: [0, 22, 47, 51], 3: [0, 9, 34, 51], 4: [0, 7, 32, 51], 5: [0, 8, 33, 51], 6: [0, 21, 46, 51], 7: [0, 15, 40, 51], 8: [0, 23, 48, 51]}


## Définition des contraintes 

### 1- Initialisation des contraintes

In [15]:
def AA(s,chauffeur):         #the arrival time of a vehicle at vertex
    Trajet=s[chauffeur]
    A={}
    d=shifts[chauffeur-1]['jobs'][0]['timeDate']
    A={0: d}
    for i in range (1,len(Trajet)):
        d+=time[Trajet[i-1],Trajet[i]]
        A[Trajet[i]]=d
    return A

def BB(s,chauffeur,i): #beginning of service at vertex i
    if i in s[chauffeur]:
        B=max(e[i],AA(s,chauffeur)[i])
        return B

def DD(s,chauffeur,i): #the departure time from vertex i
    if i!=0 and i!=2*n_req+1:
        if i<n_req+1:
            return(BB(s,chauffeur,i)-bookings[i-1]["jobs"][0]["duration"])
        else: 
            return(BB(s,chauffeur,i)-bookings[i-n_req-1]["jobs"][1]["duration"])
        
def LL(s,chauffeur,i): #The ride time associated with request i
    return (BB(s,chauffeur,i+n_req)-DD(s,chauffeur,i))

In [16]:
s=s1
#dictionaire qui contient les  temps d'arrivées à chaque noeud dans la route de chaque chaffeur k

def constrA(s):
    A={}
    for driver in s:
        A[driver]=[]
        arival=AA(s,driver)
        for u in arival :
            A[driver].append(arival[u])
    return(A)
A=constrA(s)
#dictionaire qui contient les begin time de chaque noeud dans la route de chaque chaffeur k
def constrB(s):
    B={}
    for driver in s:
        B[driver]=[]
        for i in s[driver]:
            B[driver].append(BB(s,driver,i))
    return(B)
B=constrB(s)
#dictionaire qui contient les  temps de deppart de chaque noeud dans la route de chaque chaffeur k
def constrD(s):
    D={}
    for driver in s:
        D[driver]=[A[driver][0]]
        for i in s[driver]:
            if i!=0 and i!=2*n_req+1:
                D[driver].append(DD(s,driver,i))
        D[driver].append(A[driver][-1])
    return(D)
D=constrD(s)
#dictionaire qui contient les  temps d'attente de chaque noeud dans la route de chaque chaffeur k
def constrW(s,B,A):
    W={}
    for driver in s:
        W[driver]=[]
        for i in range(len(s[driver])):
            W[driver].append(B[driver][i]-A[driver][i])
    return W
W=constrW(s,B,A)
            
        

### 2- Ecriture des contraintes

In [17]:
def pos(x):
    return(max(x,0))

def load_voilation(s,chauffeur): #violation of load cost   #s={k:[0,i,....,i+n_req,51]}
    v=0
    Q=int(shifts[chauffeur-1]["capacity"])
    cte=0
    Trajet=s[chauffeur]
    for i in Trajet:
        cte+=load[i]
        if cte>Q:
            v+=cte-Q
    return v

def window_contrainte(B,chauffeur,i): #Contrainte sur la fenêtre de temps
    S=0
    S+=pos(B[chauffeur][i]-l[s[chauffeur][i]])
    #S+=pos(BB(s,chauffeur,i)-l[i])
    return (S)

def time_contrainte(s,chauffeur,i): #violation of time constraints
    S=0
    noeud_depart=s[chauffeur][i]
    if noeud_depart<n_req and noeud_depart!=0 :
  
        noeud_arrive=n_req+noeud_depart
        i_arrive=s[chauffeur].index(noeud_arrive)
        ll=B[chauffeur][i_arrive]-D[chauffeur][i]
        S+= pos(ll-bookings[noeud_depart-1]["maximumDuration"])
        return(S)
    else:
        return(0)
        
def salary(s,chauffeur): #salary constaraint
    max=shifts[0]["maximumTurnover"]
    R=0
    S=0
    for i in s[chauffeur]:
        if i<26 and i!=0 :
            S+= bookings[i-1]['price']
        if S>max:
            R+=S-max
    return R

In [18]:
def verification_contraintes(s,chauffeur,k): #pour verifier la faisabilité des mouvements d'insertion 
    
    if load_voilation(s,chauffeur) != 0:
        return False
    
    if salary(s,chauffeur) != 0:
        return False
    
    return True

## Insertion Router - Allow Unassigned (IRAU)

<section> Afin d'ameliorer notre initiale en inserant des courses pour chaque chauffeur tout en minimisant le coût de route, nous allons procéder avec un algorithme d'insertion qui permet d'insérer à chaque itération des courses dans la position qui minimise le coût de route si ceci est possible, sinon cette commande sera stocker dans une liste des commandes non servies. <section>

### 1- Définition des mouvements possibles 

Ces fonctions vont nous servir dans le processus d'insertion, afin de déplacer les horaires du départ et du commencement de service pour chaque noeud impacter par l'inserion de le nouvelle commande dans la route du chauffeur.

In [19]:
def aMove(s,j,i,chauffeur,shift,B,D,W):
    B[chauffeur][j] = B[chauffeur][j] - shift  
    D[chauffeur][j] = B[chauffeur][j] + bookings[0]["maximumDuration"]
    u = j+1
    A=constrA(s)
    while u <= i :
        B[chauffeur][u] = B[chauffeur][u] - shift
        W[chauffeur][u] = B[chauffeur][u] - A[chauffeur][u]
        D[chauffeur][u] = B[chauffeur][u] + bookings[0]["maximumDuration"]
        u = u + 1 
    return(B,D,W)
        
def bMove(s,j,i,chauffeur,shift,B,D,W):
    B[chauffeur][i] = B[chauffeur][i] + shift
    D[chauffeur][i] = B[chauffeur][i] + bookings[0]["maximumDuration"]
    o = i+1
    A=constrA(s)
    while o <= j:
        B[chauffeur][o] = B[chauffeur][o] + shift
        W[chauffeur][o] = B[chauffeur][o] - A[chauffeur][o]
        D[chauffeur][o] = B[chauffeur][o] + bookings[0]["maximumDuration"]
        o += 1 
    return (B,D,W)
def cMove(s,i,j,chauffeur,shift,B,D,W):
    B[chauffeur][i] = B[chauffeur][i] + shift
    D[chauffeur][i] = B[chauffeur][i] + bookings[0]["maximumDuration"]
    v = i+1
    A=constrA(s)
    while v <= j and v!=len(s[k]):
        if s[chauffeur][v] <= n_req:
            B[chauffeur][v] = max(A[chauffeur][v],B[chauffeur][v])

        else:
            B[chauffeur][v] = max(A[chauffeur][v],e[s[chauffeur][v]])
            
        W[chauffeur][v] = B[chauffeur][v] - A[chauffeur][v]
        D[chauffeur][v] = B[chauffeur][v] + bookings[0]["maximumDuration"]
        v = v + 1 
    return (B,D,W)

### 2- Définition de la fonction de coût

In [20]:
#def cost#(s,k): #Total routing cost
#    c=0
#    for l in s:
#        for i in range(len(s[l])-1):
#            c+=time[(s[l][i],s[l][i+1])]
#    return c
#

In [21]:
def cost(s,k):
    c=0
    for i in s[k]:
        if i!=2*n_req+1:
            c+=time[(i,i+1)]
    return c
        

### 3- Définition des 3 cas d'insertion possibles 

In [22]:
#Definition des variables intermédiaire

def Pplus(s,k,i):
    P=[]
    for j in range(i+1):
        P.append(l[s[k][j]]-B[k][j])
    p=min(u for u in P)
    return p
    
        
def Pmoins(s,k,i):
    P=[]
    for j in range(i+1):
        P.append(B[k][j]-e[s[k][j]])
    p=min(u for u in P)
    return p


def Splus(s,k,i):
    
    L=[]
    for j in range(i,len(B[k])):
        somme=0
        for p in range(i+1,j+1):
            somme+=W[k][p]
        L.append(somme+l[s[k][j]]-B[k][j])
    if L != []:
        return min(u for u in L)
    return 0
def Smoins(s,k,i):
    P=[]
    for j in range(i,len(B[k])):
        P.append(B[k][j]-e[s[k][j]])
    p=min(u for u in P)
    return p

            

In [23]:
print( B[1][len(s[1])-1]-e[s[1][len(s[1])-1]])

17059.0


In [24]:
dd=bookings[i-1]["jobs"][0]["duration"]

In [25]:
def AI(s2,i,k,q,d,b,w):
    B=copy.copy(b)
    D=copy.copy(d)
    W=copy.copy(w)
    s=copy.copy(s2)
    s[k]=s[k][:i]+[q]+s[k][i:]
    D[k]=D[k][:i]+[B[k][i+1]-time[(q,s[k][i])]]+D[k][i:]
    A=constrA(s)
    #D[k][i]=B[k][i+1]-time[(q,s[k][i])]
    B[k]=B[k][:i]+[D[k][i]-bookings[i-1]["jobs"][0]["duration"]]+B[k][i:]
    #B[k][i]=D[k][i]-bookings[i-1]["jobs"][0]["duration"]
    D[k][0]=B[k][i]-time[(0,i)]
    qq=len(B[k])-2
    
    if verification_contraintes(s,k,i):

        if window_contrainte(B,k,i)>0: #Is the time window of si violated? yes
            if B[k][i]<e[i]:#yes
                viol=e[i]-B[k][i]
                shift1=min(viol, Splus(s,k,i))
                B,D,W=cMove(s,i,qq,k,shift1,B,D,W)
                D[k][0],B[k][qq+1] = D[k][0]+shift1,B[k][qq+1]+shift1
                if window_contrainte(B,k,i)>0: #Is the time window of si violated? yes
                    return (np.inf,s1,d,b,w)
                else:
                    return (cost(s,k),s,D,B,W)

            else: #no
                viol=B[k][i]-l[s[k][i]]
                shift1=min(viol, Smoins(s,k,i))
                B,D,W=aMove(s,i,qq,k,shift1,B,D,W)
                D[k][0],B[k][qq+1] = D[k][0]-shift1,B[k][qq+1]-shift1
               
                if window_contrainte(B,k,i)>0: #Is the time window of si violated? yes
                    shift2=pos(viol-shift1)
                    B[k][i]=B[k][i]-shift2
                    D[k][0]=D[k][0]-shift2
                    D[k][i]=D[k][i]-shift2
                    W[k][i]-=shift2
                    W[k][i+1]-=shift2
                    
                    if window_contrainte(B,k,i)>0: #Is the time window of si violated? yes:
                        return(np.inf,s1,d,b,w)
                    else:
                        return(cost(s,k),s,D,B,W)
                else:
                    return (np.inf,s1,d,b,w)         
        else: #Is the time window of si violated? no
            return(cost(s,k),s,D,B,W)
        
    else:
        return (np.inf,s1,d,b,w)


###### test AI

In [26]:
q=2
AI(s1,1,5,q,D,B,W)

(3651.0,
 {1: [0, 19, 44, 51],
  2: [0, 22, 47, 51],
  3: [0, 9, 34, 51],
  4: [0, 7, 32, 51],
  5: [0, 2, 8, 33, 51],
  6: [0, 21, 46, 51],
  7: [0, 15, 40, 51],
  8: [0, 23, 48, 51]},
 {1: [45000, 46032.0, 47090.0, 48259.0],
  2: [31200, 32640, 32778.0, 33749.0],
  3: [52200, 53640, 53640, 54085.0],
  4: [36600, 41964, 41964, 38635.0],
  5: [49622.0, 50524.0, 50032, 50464.0, 51531.0],
  6: [34800, 36240, 36240, 36885.0],
  7: [48600, 49253.0, 50019.0, 51112.0],
  8: [31200, 35340, 35340, 33306.0]},
 {1: [45000, 46092.0, 47150.0, 48259.0],
  2: [31200, 32700, 32838.0, 33749.0],
  3: [52200, 53700, 53700, 54085.0],
  4: [36600, 42024, 42024, 38635.0],
  5: [48600, 50464.0, 50092, 50524.0, 51531.0],
  6: [34800, 36300, 36300, 36885.0],
  7: [48600, 49313.0, 50079.0, 51112.0],
  8: [31200, 35400, 35400, 33306.0]},
 {1: [0, 0.0, 0.0, 0.0],
  2: [0, 636.0, 0.0, 0.0],
  3: [0, 1371.0, 467.0, 0.0],
  4: [0, 5038.0, 4496.0, 0.0],
  5: [0, 400.0, 0.0, 0.0],
  6: [0, 1022.0, 448.0, 0.0],
  7: [

In [27]:
def foctionAII(s,k,i,B,D,W):
    if verification_contraintes(s,k,i):
            if window_contrainte(s,k,i)>0:#Is the time window of si violated? yes
                if B[k][i]<e[s[k][i]]:
                    viol=e[s[k][i]]-B[k][i]
                    shift3=min(viol,Pplus(s,k,s[k][-2]))
                    B,D,W=bMove(s,1,len(s[k])-2,k,shift3,B,D,W)
                    D[k][0],B[k][-1] = D[k][0]+shift3,B[k][-1]+shift3
                    if verification_contraintes(s,k,i)== True:
                        if window_contrainte(s,k,i)>0 :#Is the time window of si violated? yes
                            shift4=min(viol-shift3,Splus(s,k,i))
                            B,D,W=cMove(s,i,len(s[k])-2,k,shift4,B,D,W)
                            B[k][-1] = B[k][-1]+shift4

                            if window_contrainte(s,k,i)>0 and verification_contraintes(s,k,i)!= True:#Is the time window of si violated? yes
                                return(np.inf,s,D,B,W)
                            else:
                                return(cost(s,k),s,D,B,W)

                        else:
                            return(cost(s,k),s,D,B,W)
                    else :
                        return(np.inf,s,D,B,W)
                else:
                    viol=B[k][i]-l[s[k][i]]
                    shift3=min(viol,Smoins(s,k,1))
                    B[k][-1]-=shift3
                    B,D,W=aMove(s,1,len(s[k])-2,k,shift3,B,D,W)
                    D[k][0] = D[k][0]-shift3
                    if window_contrainte(s,k,i)>0 and verification_contraintes(s,k,i)!= True:#Is the time window of si violated? yes
                        if s[k][i+1]>n_req: #is Si+1 a destination stop yes
                                return(np.inf,s,D,B,W)
                        else:

                            shift4=min(viol-shift3,Pmoins(s,k,i-1))
                            B,D,W=aMove(s,1,i-1,k,shift4,B,D,W)
                            D[k][0],B[k][i] = D[k][0]-shift4,B[k][i]-shift4
                            D[k][i],W[k][i],W[k][i+1] = D[k][i]-shift4,W[k][i]-shift4,W[k][i+1]-shift4

                            if window_contrainte(s,k,i)>0 and verification_contraintes(s,k,i)!= True:#Is the time window of si violated? yes
                                return(np.inf,s,D,B,W)
                            else:
                                return(cost(s,k),s,D,B,W)

                    else:
                        return(cost(s,k),s,D,B,W)

            else: #Is the time window of si violated? no
                return(cost(s,k),s,D,B,W)
    else:
        return (np.inf,s,D,B,W)

In [28]:
def AII(s2,i,k,q,d,b,w):
    B=copy.copy(b)
    D=copy.copy(d)
    W=copy.copy(w)
    s=copy.copy(s2)
    s[k]=s[k][:i]+[q]+s[k][i:]
    A=constrA(s)
    noeud=s[k][i-1]
    if noeud<= n_req:
        dd=bookings[noeud]["jobs"][0]["duration"]
    if noeud> n_req:
        dd=bookings[n_req-noeud]["jobs"][1]["duration"]
    dur=time[(s[k][i-1],q)]+dd+time[(q,s[k][i+1])]-time[(s[k][i-1],s[k][i+1])]-(B[k][i+1]-D[k][i-1])
    if dur<=0:#yes
        if s[k][i]<=n_req: #Is si an origin stop? yes
            D[k]=D[k][:i]+[B[k][i+1]-time[(q,s[k][i+1])]]+D[k][i:]
            B[k]=B[k][:i]+[D[k][i]+dd]+B[k][i:]
            W[k]=W[k][:i]+[B[k][i]-A[k][i]]+W[k][i:]
            W[k][i+1]=B[k][i+1]-A[k][i+1]
            (costt,s1,D1,B1,W1)=foctionAII(s,k,i,B,D,W)
            return (costt,s1,D1,B1,W1)
            
            
        else: #si est un depot
            B[k]=B[k][:i]+[D[k][i-1]+time[(s[k][i-1],q)]]+B[k][i:]
            D[k]=D[k][:i]+[B[k][i]+dd]+D[k][i:]
            W[k][i]=W[k][:i]+[0]+W[k][i:]
            W[k][i+1]=B[k][i+1]-A[k][i+1]
            (costt,s1,D1,B1,W1)=foctionAII(s,k,i,B,D,W)
            return (costt,s1,D1,B1,W1)
    
    
    else:
        if dur<=Pmoins(s,k,i-1)+Splus(s,k,i+1): #yes
            shift1=min(dur,Splus(s,k,i+1))
            B,D,W=cMove(s,i+1,len(s[k])-2,k,shift1,B,D,W)
            #D[k][i]=B[k][i+1]-time[(q,s[k][i])]
            D[k]=D[k][:i]+[B[k][i+1]-time[(q,s[k][i+1])]]+D[k][i:]
            B[k]=B[k][:i]+[D[k][i]-dd]+B[k][i:]
            W[k]=W[k][:i]+[B[k][i]-A[k][i]]+W[k][i:]
            B[k][len(B[k])-1]+=shift1
            
            if shift1<dur:
                shift2=dur-shift1
                B,D,W=aMove(s,1,i-1,k,shift2,B,D,W)
                D[k][0],W[k][i] = D[k][0]+shift2,W[k][i]+shift2
                (costt,s1,D1,B1,W1)=foctionAII(s,k,i,B,D,W)
                return (costt,s1,D1,B1,W1)
            else :
                (costt,s1,D1,B1,W1)=foctionAII(s,k,i,B,D,W)
                return (costt,s1,D1,B1,W1)
        else:
            return(np.inf,s2,d,b,w)
                
   

In [29]:
q=2
AII(s1,2,5,q,D,B,W)

(3651.0,
 {1: [0, 19, 44, 51],
  2: [0, 22, 47, 51],
  3: [0, 9, 34, 51],
  4: [0, 7, 32, 51],
  5: [0, 8, 2, 33, 51],
  6: [0, 21, 46, 51],
  7: [0, 15, 40, 51],
  8: [0, 23, 48, 51]},
 {1: [45000, 46032.0, 47090.0, 48259.0],
  2: [31200, 32640, 32778.0, 33749.0],
  3: [52200, 53640, 53640, 54085.0],
  4: [36600, 41964, 41964, 38635.0],
  5: [48600, 50032, 50778.0, 50464.0, 51531.0],
  6: [34800, 36240, 36240, 36885.0],
  7: [48600, 49253.0, 50019.0, 51112.0],
  8: [31200, 35340, 35340, 33306.0]},
 {1: [45000, 46092.0, 47150.0, 48259.0],
  2: [31200, 32700, 32838.0, 33749.0],
  3: [52200, 53700, 53700, 54085.0],
  4: [36600, 42024, 42024, 38635.0],
  5: [48600, 50092, 50838.0, 50524.0, 51531.0],
  6: [34800, 36300, 36300, 36885.0],
  7: [48600, 49313.0, 50079.0, 51112.0],
  8: [31200, 35400, 35400, 33306.0]},
 {1: [0, 0.0, 0.0, 0.0],
  2: [0, 636.0, 0.0, 0.0],
  3: [0, 1371.0, 467.0, 0.0],
  4: [0, 5038.0, 4496.0, 0.0],
  5: [0, 400.0, 215.0, -852.0, 0.0],
  6: [0, 1022.0, 448.0, 0.0]

In [30]:
def AIII(s1,i,k,q,d,b,w):
    B=copy.copy(b)
    D=copy.copy(d)
    W=copy.copy(w)
    s=copy.copy(s1)
    A=constrA(s)
    s[k]=s[k][:i]+[q]+s[k][i:]
    Cost = 0
    q1 = len(s[k])-2
    B[k] = B[k][:i] + [D[k][i-1] + Time[(s[k][i-1],s[k][i])]] + B[k][i:]
    D[k] = D[k][:i] + [B[k][i] + bookings[i-1]["maximumDuration"]] + D[k][i:]
    D[k][q1+1] = B[k][i] + time[(s[k][i],s[k][q1+1])]
    
    
    if window_contrainte(s,k,i) == 0 and verification_contraintes(s,k,i) == True: #Is the time window of Si violated ? --> Non
        Cost = cost(s,k)
        return (Cost,s,D,B,W)
    
    elif B[k][i] < e[s[k][i]]:              #Is Bi < ei ? --> Oui
        viol = e[s[k][i]] - B[k][i]
        shift1 = min(viol, Pplus(s,k,i))
        B,D,W=bMove(s,1,q1,k,shift1,B,D,W)
        D[k][0], B[k][q1+1] = D[k][0] - shift1, B[k][q1+1] - shift1 #Update
        
        if window_contrainte(s,k,i) != 0 or verification_contraintes(s,k,i) == False: #Is the time window of "Si" violated ? --> Oui
            if i > n_req:              #Is "Si" a destination stop ?
                return (np.inf,s1,d,b,w)
            else: 
                shift2 = viol - shift1
                B[k][i] = B[k][i] + shift2
                D[k][i], W[k][i], B[k][q1+1] = D[k][i] + shift2, W[k][i] + shift2, B[k][q1+1] + shift2 #Update
                
                if window_contrainte(s,k,i) != 0 or verification_contraintes(s,k,i) == False: #Is the time window of "Si" violated ? --> Oui
                    return (np.inf,s1,d,b,w)
                else: 
                    Cost = cost(s,k)
                    return (Cost)       
        else:                        #Is the time window of "Si" violated ? --> Non
            Cost = cost(s,k)         
            return (Cost,s,D,B,W)
    else:                            #Is Bi < ei ?  --> Non
        viol = B[k][i] - l[s[k][i]]
        shift1 = min(viol, Pmoins(s,k,i-1))
        B,D,W=aMove(s,1,i,k,shift1,B,D,W)
        D[k][0], B[k][q1+1] = D[k][0] + shift1, B[k][q1+1] + shift1 #Update
        
        if window_contrainte(s,k,i) != 0 or verification_contraintes(s,k,i) == False: #Is the time window of Si violated ? --> Oui
            return (np.inf,s1,d,b,w)
        else:
            Cost = cost(s,k)
            return (Cost,s,D,B,W)

In [31]:
def BI(s,i,k,q,D,B,W):
    (cost1,s1,D1,B1,W1)=AI(s,i,k,q,D,B,W)
    if cost1==np.inf:
        return(np.inf,s,D,B,W)
    else :
        kprim=i+1
        #print(k,AII(s1,kprim,k,q,D1,B1,W1))
        (cost2,s2,D2,B2,W2)=AII(s1,kprim,k,q,D1,B1,W1)
        if cost2==np.inf:
            return(np.inf,s,D,B,W)
        else:
            coost=float(cost1+cost2)
            return(coost,s2,D2,B2,W2)
def BII(s,i,k,q,D,B,W):
    (cost1,s1,D1,B1,W1)=AII(s,i,k,q,D,B,W)
    if cost1==np.inf:
        return(np.inf,s,D,B,W)
    else :
        kprim=i+1
        (cost2,s2,D2,B2,W2)=AII(s1,kprim,k,q,D1,B1,W1)
        if cost2==np.inf:
            return(np.inf,s,D,B,W)
        else:
            coost=float(cost1+cost2)
            return(coost,s2,D2,B2,W2)
def BIII(s,i,k,q,D,B,W): 
    (cost1,s1,D1,B1,W1)=AII(s,i,k,q,D,B,W)
    if cost1==np.inf:
        return(np.inf,s,D,B,W)
    else :
        kprim=i+1
        (cost2,s2,D2,B2,W2)=AIII(s1,kprim,k,q,D1,B1,W1)
        if cost2==np.inf:
            return(np.inf,s,D,B,W)
        else:
            coost=float(cost1+cost2)
            return(coost,s2,D2,B2,W2)

### 4- Algorithme d'insertion:

##### 4.1- Liste des commandes à attribuer

In [32]:
#les noeuds non attribuées à un chauffeur
s=s1
NU=[i for i in range(1,n_req+1)]
for k in s:
     for j in s[k]:
        if 0<j and j<=n_req:
            NU.remove(j)

##### 4.2- fonctions de récuperation des meilleurs coûts:

In [33]:
def meilleurposition(costk):
    a=np.inf
    j=1
    for i in range(len(costk)):
        if costk[i]<a:
            j=i+1
            a=costk[i]
    return(j,a)
def Meilleurcost(costq):
    a=np.inf
    k=0
    posi=1
    for ii in range(len(costq)):
        if costq[ii][1]<a:
            k=ii
            a=costq[ii][1]
            posi=costq[ii][0]
    return(k,posi,a)
        

##### 4.3- Algorithme 

In [34]:
def Insertion(s,NU,D,B,W):
    NF=[] #solution non faisable
    A=constrA(s)
    s1=copy.copy(s)
    for q in NU:
        costq=[]
        for k in s:
            costk=[]
            for i in range(len(s[k])):
                if i==1:
                    costk.append(BI(s,i,k,q,D,B,W)[0])
                elif 1<i and i<len(s[k])-2:
                    costk.append(BII(s,i,k,q,D,B,W)[0])
                elif i==len(s[k])-2:
                    costk.append(BIII(s,i,k,q,D,B,W)[0])
                    
                
        #trouver la meilleure position d'insertion
            (position,meilleurcost)=meilleurposition(costk)
            costq.append([position,meilleurcost])
        chauf,pos,costt=Meilleurcost(costq)
        if costt==np.inf:
            NF.append(q)
            
        else:
            
            s[chauf+1]=s1[chauf+1][:pos]+[q,n_req+q]+s1[chauf+1][pos:]
            print(s)
            A=constrA(s)
            D=constrD(s)
            B=constrB(s)
            W=constrW(s,B,A)
            
    
    return(s,NF)
    

In [35]:
Insertion(s1,NU,D,B,W)

{1: [0, 19, 44, 51], 2: [0, 22, 47, 51], 3: [0, 9, 34, 51], 4: [0, 7, 32, 51], 5: [0, 8, 33, 51], 6: [0, 21, 46, 51], 7: [0, 15, 1, 26, 40, 51], 8: [0, 23, 48, 51]}
{1: [0, 19, 44, 51], 2: [0, 22, 47, 51], 3: [0, 3, 28, 9, 34, 51], 4: [0, 7, 32, 51], 5: [0, 8, 33, 51], 6: [0, 21, 46, 51], 7: [0, 15, 1, 26, 40, 51], 8: [0, 23, 48, 51]}
{1: [0, 19, 44, 51], 2: [0, 22, 4, 29, 47, 51], 3: [0, 3, 28, 9, 34, 51], 4: [0, 7, 32, 51], 5: [0, 8, 33, 51], 6: [0, 21, 46, 51], 7: [0, 15, 1, 26, 40, 51], 8: [0, 23, 48, 51]}
{1: [0, 19, 44, 51], 2: [0, 22, 4, 29, 47, 51], 3: [0, 3, 28, 9, 34, 51], 4: [0, 7, 32, 51], 5: [0, 8, 33, 51], 6: [0, 21, 5, 30, 46, 51], 7: [0, 15, 1, 26, 40, 51], 8: [0, 23, 48, 51]}


IndexError: list index out of range

In [36]:
s_final={1: [0, 19, 44, 51], 2: [0, 22, 4, 29, 47, 51], 3: [0, 3, 28, 9, 34, 51], 4: [0, 7, 32, 51], 5: [0, 8, 33, 51], 6: [0, 21, 5, 30, 46, 51], 7: [0, 15, 1, 26, 40, 51], 8: [0, 23, 48, 51]}

In [38]:
for k in s_final:
    for i in s_final[k]:
        print(verification_contraintes(s_final,k,i))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
